In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.ensemble import RandomForestRegressor
import pickle
from mosaicml import *
from mosaicml.constants import MLModelFlavours

In [2]:
df = pd.read_csv("/data/winequality.csv")
df.head(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
5,7.4,0.66,0.00,1.8,0.075,13.0,40.0,0.9978,3.51,0.56,9.4,5
6,7.9,0.60,0.06,1.6,0.069,15.0,59.0,0.9964,3.30,0.46,9.4,5
7,7.3,0.65,0.00,1.2,0.065,15.0,21.0,0.9946,3.39,0.47,10.0,7
8,7.8,0.58,0.02,2.0,0.073,9.0,18.0,0.9968,3.36,0.57,9.5,7
9,7.5,0.50,0.36,6.1,0.071,17.0,102.0,0.9978,3.35,0.80,10.5,5


In [3]:
# The target variable is 'quality'.
Y = df['quality']
X = df[['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar','chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density','pH', 'sulphates', 'alcohol']]


# Split the data into train and test data:
from sklearn.model_selection import train_test_split
import os

#split_percentage = float(os.getenv("split_percentage"))
split_percentage = 0.2
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = split_percentage)
X_train = X_train.to_numpy()
Y_train = Y_train.to_numpy()


# Use ML Algorithm
from sklearn.ensemble import RandomForestRegressor

my_model = RandomForestRegressor(max_depth=6, random_state=0, n_estimators=10)
my_model.fit(X_train,Y_train)

@scoring_func
def score(model, request):
    payload = request.json["payload"]
    data_list = payload
    data_array = np.asarray(data_list)
    try:
        prediction = model.predict(data_array)
    except:
        prediction = model.predict(data_array.reshape(1, -1))
    return prediction.tolist()

In [4]:
import requests
req = requests.Request()
req.json = {"payload":[ 7.4   ,  0.35  ,  0.33  ,  2.4   ,  0.068 ,  9    , 26    ,
        0.9947,  3.36  ,  0.6   , 11.9   ]}

score(my_model,req)

[6.041562329737387]

In [6]:
register_model(my_model,
                score,
                "wine_quality_Py_37",
                "wine_quality_regression_notebook",
                MLModelFlavours.sklearn,
                init_script="mkdir abc \\n cd abc \\n touch abc.txt",
                input_type="json",
                explain_ai=True,
                x_train=X_train, 
                y_train=Y_train, 
                feature_names=['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar','chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density','pH', 'sulphates', 'alcohol'], 
                feature_ids=['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar','chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density','pH', 'sulphates', 'alcohol'],
                target_names=["wine_quality"]
              )

Exception: Unable to register model. Please try after some time